<a href="https://colab.research.google.com/github/ArtemJDS/NN-for-population-approximation/blob/main/Koopman_autoencoder_for_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import h5py
import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from google.colab import files
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import normalize
tf.keras.backend.set_floatx('float32')
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/voltage_and_input.zip

In [ ]:
dataset = h5py.File('/content/content/voltage_and_input.h5', 'r')
voltage = np.array(dataset['voltage'], dtype = np.float32).T    # population's voltage dynamics               
input = np.array(dataset['input'], dtype = np.float32)     # population's input   

min = np.min(voltage)
max = np.max(voltage)
bins = np.linspace(min, max, N_BINS)     
x = np.apply_along_axis(np.histogram, 1, voltage, bins = bins, density = True).T[0]
x = list(x)
x = np.stack(x)
densities = normalize(x, norm = 'l1') # shape = n,m, where n = # of timesteps, m = N_BINS

In [ ]:
batch_size = 128
train_dataset = np.array([densities[:-5], densities[1:-4], densities[2:-3], densities[3:-2], densities[4:-1]])
train_dataset = tf.convert_to_tensor(train_dataset)
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).shuffle(1104636).batch(batch_size)

There is an attempt to apply koopmanism for population's voltage distribution approximation.

The procedure in general follows the described in "Lusch, B., Kutz, J.N. & Brunton, S.L. Deep learning for universal linear embeddings of nonlinear dynamics. Nat Commun 9, 4950 (2018)"

train_dataset contains n (here n equals 5) ordered snapshots of a system. Network's task is approximation of the system's trajectory. 

It's achieved by means of nonlinear dimensionality reduction (autoencoder). Then system is propagated linearly (Koopman operator) in the new coordinates. At each step approximated distribution gets compared with the actual one. 


In [ ]:
LATENT_DIM = 2
N_BINS = 249

class Autoencoder(Model):
    def __init__(self, latent_dim, n_bins):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.n_bins = n_bins

        self.main_encoder = tf.keras.Sequential([
                            layers.Flatten(),
                            layers.Dense(256, activation='relu', kernel_regularizer='l2'), 
                            layers.Dense(256, activation='relu', kernel_regularizer='l2'),
                            layers.Dense(self.latent_dim,activation='relu', kernel_regularizer='l2'),
                            ])
      
        self.Koopman = tf.keras.Sequential([
                            layers.Dense(self.latent_dim, use_bias=True)
                        ])

        self.decoder = tf.keras.Sequential([
                        layers.Dense(256, activation='relu', kernel_regularizer='l2'),
                        layers.Dense(256, activation='relu', kernel_regularizer='l2'), 
                        layers.Dense(self.n_bins, activation='relu', kernel_regularizer='l2'),
                        ])

    def call(self, input):

        nth_main = tf.zeros([batch_size, 2])
        encoded_main = tf.zeros([batch_size, latent_dim])
        K_activated = tf.zeros([batch_size, latent_dim])
        decoded = tf.zeros([batch_size, 2])
        eigenvalues = tf.zeros([batch_size, latent_dim])

        n_plus_one_main = nth_main
        encoded_n_plus_main = encoded_main

        loss_decode = 0.
        loss_repr = 0.
        loss_der = 0.

        input = tf.transpose(input, [1,0,2])
        l = input.shape[0] - 1
        for n in tf.range(l):

            i = input[n]
            j = input[n+1]
            if n == 0:
              tr = tf.transpose(i) 
             
              nth_main = tf.reshape(tf.transpose(tr[:]), [-1,2])
              encoded_main = self.main_encoder(nth_main)
              K_activated = self.Koopman(encoded_main)
              decoded = self.decoder(encoded_main)  

              tr_new = tf.transpose(j)
              n_plus_one_main = tf.reshape(tf.transpose(tr_new[:]), [-1,2])
              encoded_n_plus_main = self.main_encoder(n_plus_one_main)

              loss_decode +=  tf.math.reduce_sum(tf.math.square(nth_main - decoded))
              loss_repr += tf.math.reduce_sum(tf.math.square(K_activated - encoded_n_plus_main))
            
            if n != 0:

              K_activated = self.Koopman(K_activated)
              decoded = self.decoder(K_activated)

              tr_new = tf.transpose(j)
              n_plus_one_main = tf.reshape(tf.transpose(tr_new[:]), [-1,2])
              encoded_n_plus_main = self.main_encoder(n_plus_one_main)

              loss_decode += tf.math.reduce_sum(tf.math.square(tf.transpose(tf.transpose(n_plus_one_main)[:]) - decoded))
              loss_repr += tf.math.reduce_sum(tf.math.square(K_activated - encoded_n_plus_main))

        return loss_repr, loss_decode, loss_decode + loss_repr * 0.0005
autoencoder = Autoencoder(LATENT_DIM)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


In [ ]:
EPOCHES = 100

@tf.function
def training_step(inp):
    with tf.GradientTape() as tape:
        loss_a, loss_b, loss = autoencoder(inp)
    grads = tape.gradient(loss, autoencoder.trainable_variables)
    optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))
    return loss_a, loss_b


l = []
for n in range(EPOCHES):
        l1 = []
        l2 = []

        for inp in dataset_train: 
    
            loss1, loss2 = training_step(inp)
            l1.append(float(loss1))
            l2.append(float(loss2))

        print(f'ITERATION {n}, {round(sum(l1)/len(l1), 8)}, {round(sum(l2)/len(l2), 8)}')

        l.append(l1)
        l.append(l2)
